In [54]:
from tweebo_parser import API, ServerError
import numpy as np
import pandas as pd

import sys
import re
import string
import csv
import random
import time
import emoji
import trie
import datetime

import NE_candidate_module as ne
import Mention
# import phase2_Trie_baseline_reintroduction_effectiveness as phase2

# import twokenize

from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from scipy import stats

In [55]:
# tokens=word_tokenize("Very well explained take on Carter/ Russia/ FISA/ Trump's sitch.")
# print(tokens)

In [56]:
#---------------------Existing Lists--------------------
cachedStopWords = stopwords.words("english")
tempList=["i","and","or","other","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
cachedStopWords.remove("your")
cachedStopWords.remove("up")
cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["in","at","of","on","v."] #includes common conjunction as well
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","gee","hmm","bye","vs","ouch","omw","qt","dj","dm","congrat","haueheuaeh","ahushaush","jr","please","retweet","2mrw","2moro","4get","ooh","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]

string.punctuation=string.punctuation+'…‘’'
#string.punctuation.extend('“','’','”')
#---------------------Existing Lists--------------------

gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(gutenberg_text)
my_sentence_tokenizer = PunktSentenceTokenizer(trainer.get_params())
my_sentence_tokenizer._params.abbrev_types.add('dr')
my_sentence_tokenizer._params.abbrev_types.add('c.j')
my_sentence_tokenizer._params.abbrev_types.add('u.s')
my_sentence_tokenizer._params.abbrev_types.add('u.s.a')
my_sentence_tokenizer._params.abbrev_types.add('ret.')
my_sentence_tokenizer._params.abbrev_types.add('rep.')



In [57]:
all_stopwords=cachedStopWords+cachedTitles+prep_list+article_list+conjoiner+day_list+month_list+chat_word_list

In [58]:
# Assumes server is running locally at 0.0.0.0:8000
tweebo_api = API()
proper_noun_tag='^'
common_noun_tag='N'
prep_tag='P'


def flatten(mylist, outlist,ignore_types=(str, bytes, int)):
    
    if mylist !=[]:
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                flatten(item, outlist)
            else:
#                 if isinstance(item,ne.NE_candidate):
#                     item.phraseText=item.phraseText.strip(' \t\n\r')
#                     item.reset_length()
#                 else:
                if type(item)!= int:
                    item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist
    
def splitSentence(tweetText):
#     print(tweetText)
    tweetSentences=list(filter (lambda sentence: len(sentence)>1, tweetText.split('\n')))
    # tweetSentenceList_inter=self.flatten(list(map(lambda sentText: sent_tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList_inter= flatten(list(map(lambda sentText: my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
    return tweetSentenceList

In [59]:
def getWords(sentence):
    tempList=[]
    tempWordList=sentence.split()
    p_dots= re.compile(r'[.]{2,}')
    #print(tempWordList)
    for word in tempWordList:
        temp=[]

        if "(" in word:
            temp=list(filter(lambda elem: elem!='',word.split("(")))
            if(temp):
                temp=list(map(lambda elem: '('+elem, temp))
        elif ")" in word:
            temp=list(filter(lambda elem: elem!='',word.split(")")))
            if(temp):
                temp=list(map(lambda elem: elem+')', temp))
            # temp.append(temp1[-1])
#         elif (("-" in word)&(not word.endswith("-"))):
#             temp1=list(filter(lambda elem: elem!='',word.split("-")))
#             if(temp1):
#                 temp=list(map(lambda elem: elem+'-', temp1[:-1]))
#             temp.append(temp1[-1])
        elif (("?" in word)&(not word.endswith("?"))):
            temp1=list(filter(lambda elem: elem!='',word.split("?")))
            if(temp1):
                temp=list(map(lambda elem: elem+'?', temp1[:-1]))
            temp.append(temp1[-1])
        elif ((":" in word)&(not word.endswith(":"))):
            temp1=list(filter(lambda elem: elem!='',word.split(":")))
            if(temp1):
                temp=list(map(lambda elem: elem+':', temp1[:-1]))
            temp.append(temp1[-1])
        elif (("," in word)&(not word.endswith(","))):
            #temp=list(filter(lambda elem: elem!='',word.split(",")))
            temp1=list(filter(lambda elem: elem!='',word.split(",")))
            if(temp1):
                temp=list(map(lambda elem: elem+',', temp1[:-1]))
            temp.append(temp1[-1])
        elif (("/" in word)&(not word.endswith("/"))):
            temp1=list(filter(lambda elem: elem!='',word.split("/")))
            if(temp1):
                temp=list(map(lambda elem: elem+'/', temp1[:-1]))
            temp.append(temp1[-1])
        elif (list(p_dots.finditer(word))):
            matched_spans= list(p_dots.finditer(word)) 
            temp=[]
            next_string_start=0
            for matched_span in matched_spans:
                matched_start=matched_span.span()[0]
                this_excerpt=word[next_string_start:matched_start]
                if(this_excerpt):
                    temp.append(this_excerpt)
                next_string_start=matched_span.span()[1]
            if(next_string_start<len(word)):
                last_excerpt=word[next_string_start:]
                if(last_excerpt):
                    temp.append(last_excerpt)
        elif "…" in word:
            temp=list(filter(lambda elem: elem!='',word.split("…")))
            if(temp):
                if(word.endswith("…")):
                    temp=list(map(lambda elem: elem+'…', temp))
                else:
                    temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
        else:
            #if word not in string.punctuation:
            temp=[word]
        if(temp):
            tempList.append(temp)
    tweetWordList=flatten(tempList,[])
    return tweetWordList
    

In [60]:
def getWordsII(sentence):
    tempList=[]
    tempWordList=sentence.split()
    p_dots= re.compile(r'[.]{2,}')
    #print(tempWordList)
    for word in tempWordList:
        if (list(p_dots.finditer(word))):
#             print('==>',word)
            matched_spans= list(p_dots.finditer(word)) 
            temp=[]
            next_string_start=0
            for matched_span in matched_spans:
                matched_start=matched_span.span()[0]
                this_excerpt=word[next_string_start:matched_start]
                if(this_excerpt):
                    temp.append(this_excerpt)
                next_string_start=matched_span.span()[1]
            if(next_string_start<len(word)):
                last_excerpt=word[next_string_start:]
                if(last_excerpt):
                    temp.append(last_excerpt)
#             print(temp)
        elif((word.count('.')==1)&(word.endswith('.'))):
            words=list(filter(lambda elem: elem!='',re.split("(\.)",word)))
            temp=[]
            for token in words:
                if(token!='.'):
                    temp+=list(filter(lambda elem: elem!='',re.split('([^a-zA-Záéíó@#’0-9\'])',token)))
                else:
                    temp.append('.')
        else:
            temp=list(filter(lambda elem: elem!='',re.split('([^a-zA-Záéíó@.#’\'0-9])',word)))
        if(temp):
            tempList.append(temp)
    tweetWordList=flatten(tempList,[])
    return tweetWordList

In [61]:
def normalize(strip_op):
#     strip_op=word
    strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
    strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
    #strip_op= self.rreplace(self.rreplace(self.rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
#     if strip_op.endswith("'s"):
#         li = strip_op.rsplit("'s", 1)
#         return ''.join(li)
#     elif strip_op.endswith("’s"):
#         li = strip_op.rsplit("’s", 1)
#         return ''.join(li)
#     else:
#         return strip_op
    return strip_op

def split_apostrophe(strip_op):
    if strip_op.endswith("'s"):
#         print('==>',strip_op)
        li = strip_op.rfind("'s")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("’s"):
        li = strip_op.rfind("’s")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("'S"):
#         print('==>',strip_op)
        li = strip_op.rfind("'S")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("’S"):
#         print('==>',strip_op)
        li = strip_op.rfind("’S")
        return [strip_op[:li],strip_op[li:]]
    else:
        return [strip_op]
#     return strip_op
    
def get_encoding_seq(tweet_word_list, mentions):
    print(tweet_word_list)
    print(mentions)
    tweet_word_index=0
    encoded_tag_sequence=[]
    while(mentions):
        current_mention=[token.strip() for token in mentions.pop(0).split(' ')]
        while(normalize(current_mention[0])!=normalize(tweet_word_list[tweet_word_index])):
            encoded_tag_sequence.append('O')
            tweet_word_index+=1
        if(normalize(current_mention[0])==normalize(tweet_word_list[tweet_word_index])):
            for token_index, token in enumerate(current_mention):
                if(token_index==0):
                    encoded_tag_sequence.append('B')
                else:
                    encoded_tag_sequence.append('I')
                tweet_word_index+=1
    while(tweet_word_index<len(tweet_word_list)):
        encoded_tag_sequence.append('O')
        tweet_word_index+=1
        
    print(encoded_tag_sequence)
    return encoded_tag_sequence
        

## Just STS Data Processing not regular datasets (Dont run otherwise)

In [ ]:
def prepare_sts_data(filename):
    stsSentence1DataDictList=[]
    stsSentence2DataDictList=[]
    stsData_columns =['sentence1', 'sentence2', 'score']
    f=open(filename,'r')
    file_text= f.read()
    lines=file_text.split('\n')
    for line in lines:
        if(line):
            fields=line.split('\t')
            stsSentence1DataDictList.append(fields[5])
            stsSentence2DataDictList.append(fields[6])
    print('=>',len(stsSentence1DataDictList),len(stsSentence2DataDictList))
    return stsSentence1DataDictList, stsSentence2DataDictList

In [31]:
# stsSentence1DataList, stsSentence2DataList = prepare_sts_data("/Users/satadisha/Documents/GitHub/stsbenchmark/sts-dev.csv")

=> 1500 1500


In [32]:
# print(len(stsSentence1DataList))

1500


In [33]:
# result_conll=[]
# try:
#     result_conll = tweebo_api.parse_conll(stsSentence1DataList)
# except ServerError as e:
#     print(f'{e}\n{e.message}')
# print('sentence1 parse done!')
# print(len(stsSentence1DataList))

# f1 = open("/Users/satadisha/Documents/GitHub/sts.sentence1.dev.conll.preproc.url", "w")
# file_text1=''

# f = open("/Users/satadisha/Documents/GitHub/sts.sentence1.dev.conll.preproc.url.postag", "w")
# file_text=''

# candidates=[]
# sentId=0
# # tweetId=0
# CTrie=trie.Trie("ROOT")
# for sentence in stsSentence1DataList:
# # for tweet in tweetList:
#     result=result_conll[sentId]
# #     result=result_conll[tweetId]
#     tweet_word_list=[]
#     conll_results=[result_line.split('\t') for result_line in result.split('\n')]
#     for result_line in conll_results:
#         file_text+=result_line[1]+'\t'+result_line[3]+'\n'
#         tweet_word_list+=result_line[1]
#         file_text1+=result_line[1]+'\tO'+'\n'
# #         print('result_line: ', result_line)
#     file_text+='\n'
# #     tweetId+=1
# #     tweetMentions=mentionList[tweetId]
# #     tweet_encoding_list= get_encoding_seq(tweet_word_list, tweetMentions)
# #     print(len(tweet_word_list),len(tweet_encoding_list))
# #     for ind, word in enumerate(tweet_word_list):
# #         file_text1+=word+'\t'+tweet_encoding_list[ind]+'\n'
#     file_text1+='\n'
#     sentId+=1
# #     tweetId+=1

# f.write(file_text)
# f.close()

# f1.write(file_text1)
# f1.close()

# result_conll=[]
# try:
#     result_conll = tweebo_api.parse_conll(stsSentence2DataList)
# except ServerError as e:
#     print(f'{e}\n{e.message}')
# print('sentence2 parse done!')
# print(len(stsSentence2DataList))

# f1 = open("/Users/satadisha/Documents/GitHub/sts.sentence2.dev.conll.preproc.url", "w")
# file_text1=''

# f = open("/Users/satadisha/Documents/GitHub/sts.sentence2.dev.conll.preproc.url.postag", "w")
# file_text=''

# candidates=[]
# sentId=0
# # tweetId=0
# CTrie=trie.Trie("ROOT")
# for sentence in stsSentence2DataList:
# # for tweet in tweetList:
#     result=result_conll[sentId]
# #     result=result_conll[tweetId]
#     tweet_word_list=[]
#     conll_results=[result_line.split('\t') for result_line in result.split('\n')]
#     for result_line in conll_results:
#         file_text+=result_line[1]+'\t'+result_line[3]+'\n'
#         tweet_word_list+=result_line[1]
#         file_text1+=result_line[1]+'\tO'+'\n'
# #         print('result_line: ', result_line)
#     file_text+='\n'
# #     tweetId+=1
# #     tweetMentions=mentionList[tweetId]
# #     tweet_encoding_list= get_encoding_seq(tweet_word_list, tweetMentions)
# #     print(len(tweet_word_list),len(tweet_encoding_list))
# #     for ind, word in enumerate(tweet_word_list):
# #         file_text1+=word+'\t'+tweet_encoding_list[ind]+'\n'
#     file_text1+='\n'
#     sentId+=1
# #     tweetId+=1

# f.write(file_text)
# f.close()

# f1.write(file_text1)
# f1.close()

sentence1 parse done!
1500
sentence2 parse done!
1500


In [62]:
#3k annotated tweets
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/TwiCSv2/data/tweets_3k_annotated.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/venezuela.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/billnye.csv",sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/pikapika.csv",sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/ripcity.csv",sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/roevwade.csv",sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/billdeblasio.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/wnut17test.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/covid_2K.csv",sep =',',keep_default_na=False) 

tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/broad_twitter_corpus.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/all-input-files/deduplicated_test.csv",sep =';',keep_default_na=False)

# tweets=tweets_unpartitoned['TweetText'].tolist()
# print(len(tweets_unpartitoned))


In [63]:
# tweets_1=pd.read_csv("/Users/satadisha/Documents/GitHub/billnye.csv",sep =',',keep_default_na=False)
# tweets_2=pd.read_csv("/Users/satadisha/Documents/GitHub/pikapika.csv",sep =',',keep_default_na=False)
# tweets_3=pd.read_csv("/Users/satadisha/Documents/GitHub/ripcity.csv",sep =',',keep_default_na=False)

# tweets_unpartitoned= pd.concat([tweets_1,tweets_2,tweets_3])
print(len(tweets_unpartitoned))
print(tweets_unpartitoned.columns)

9338
Index(['ID', 'HashTags', 'TweetText', 'Output', 'mentions_other', 'User',
       'ritter_candidates', 'calai_candidates', 'stanford_candidates'],
      dtype='object')


# with TurboParser NP Chunker:

In [64]:
npchunker_phaseI_time_in = time.time()

In [65]:
df_holder=[]
batch_number=0
# tweetList=[]
sentenceList=[]
sentID=0
sentID_to_tweet_ID={}
mentionList=[]

for row in tweets_unpartitoned.itertuples():

    index=row.Index
    
    now = datetime.datetime.now()
    hashtags=str(row.HashTags)

    user=str(row.User)
    tweetText=str(row.TweetText)
    annot_raw=""
    stanford_candidates=""
    ritter_candidates = ""
    calai_candidates=""

    ne_List_final=[]
    userMention_List_final=[]
    tweetSentenceList=splitSentence(tweetText)
    sentenceList.extend(tweetSentenceList)
    
    for sentence in tweetSentenceList:
        sentID_to_tweet_ID[sentID]=int(index)
        sentID+=1
    
    mentions=[]
    
#     if(len(tweetSentenceList)!=len(str(row.mentions_other_BIO).split(';'))):
#         print('index: ',index)

##     for datasets that are not annotated, comment this out
#     for sentence_level in str(row.mentions_other).split(';'):
#         if(sentence_level):
#             for mention in sentence_level.split(','):
#                 if(mention):
#                     mentions.append(mention.strip())
    
#     if(len(tweetSentenceList)!= len(mentions)):
#         print('tally: ',len(tweetSentenceList), len(mentions))
#         print(tweetSentenceList)
#         print(row.mentions_other)
#     print(mentions)

    mentionList.append(mentions)
#     tweetList.append(tweetText)
    
    for sen_index in range(len(tweetSentenceList)):
        sentence=tweetSentenceList[sen_index]
        annotation=[]
        tweetWordList=getWords(sentence)
        enumerated_tweetWordList=[(token,idx) for idx,token in enumerate(tweetWordList)]
#         phase1Candidates
        dict1 = {'tweetID':str(index), 'sentID':str(sen_index), 'hashtags':hashtags, 'user':user, 'TweetSentence':sentence, 'tweetwordList': enumerated_tweetWordList, 'start_time':now,'entry_batch':batch_number,'annotation':annotation,'stanford_candidates':stanford_candidates,'ritter_candidates':ritter_candidates,'calai_candidates':calai_candidates}
        df_holder.append(dict1)

#     for candidate in ne_List_final:
#         #self.insert_dict (candidate,self.NE_container,candidateBase,index,candidate.sen_index,batch_number)
#         candidateText=(((candidate.phraseText.lstrip(string.punctuation)).rstrip(string.punctuation)).strip(' \t\n\r')).lower()
#         candidateText=(candidateText.lstrip('“‘’”')).rstrip('“‘’”')
#         candidateText= self.rreplace(self.rreplace(self.rreplace(candidateText,"'s","",1),"’s","",1),"’s","",1)
#         # if(index==9423):
#         #     print(candidateText)
#         combined=[]+cachedStopWords+cachedTitles+prep_list+chat_word_list+article_list+day_list
#         if not ((candidateText in combined)|(candidateText.isdigit())|(self.is_float(candidateText))):
#             self.CTrie.__setitem__(candidateText.split(),len(candidateText.split()),candidate.features,batch_number)

#     NE_list_phase1+=ne_List_final

#     UserMention_list+=userMention_List_final

tweet_sentence_df= pd.DataFrame(df_holder,columns=('tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence','tweetwordList', 'start_time','entry_batch','annotation','stanford_candidates','ritter_candidates','calai_candidates'))
print(len(sentenceList))

15036


In [66]:
result_conll=[]
try:
# #     result_stanford = tweebo_api.parse_stanford(text_data)
# #     result_conll = tweebo_api.parse_conll(text_data)
    result_conll = tweebo_api.parse_conll(sentenceList)
# #     result_conll += tweebo_api.parse_conll(tweetList[:1000])
# #     result_conll += tweebo_api.parse_conll(tweetList[1000:2000])
# #     result_conll += tweebo_api.parse_conll(tweetList[2000:3000])
# #     result_conll += tweebo_api.parse_conll(tweetList[3000:])
except ServerError as e:
    print(f'{e}\n{e.message}')

In [67]:
print('parse done!')
print(len(sentenceList))
# print(len(tweetList))
# print(len(result_conll))

# print(sentID_to_tweet_ID[15])
# print(result_conll[15])

parse done!
15036


In [41]:
# # just printing the twokenized sentences
# sentId=0
# df_holder=[]
# df_columns=['tweet_id','sentence_id','word']
# for sentence in sentenceList:
# #     print(sentence)
#     sentence_tokens= flatten([split_apostrophe(elem) for elem in getWordsII(sentence)],[])
# #     print(sentence_tokens)
# #     result=result_conll[sentId]
#     for token in sentence_tokens:
# #     for result_line in result.split('\n'):
# #         tabs = result_line.split('\t')
#         df_dict={'tweet_id':str(sentID_to_tweet_ID[sentId]),'sentence_id':str(sentId), 'word':token}
#         df_holder.append(df_dict)
#     sentId+=1

# df_out = pd.DataFrame(df_holder,columns=df_columns)
# print('pre-encoding dataframe: ', len(df_out))

# #align mentions with tweets and generate BIO encoding:
# encoded_df_columns=['Sentence #','Word','Tag']
# encoded_df_holder=[]

# # file_text=''
# for index, mentions in enumerate(mentionList):
#     tweet_sentID_list= df_out[df_out['tweet_id']==str(index)].sentence_id.tolist()
#     tweet_word_list= df_out[df_out['tweet_id']==str(index)].word.tolist()
#     print('tweet ID:', index,mentions)
#     tweet_encoding_list= get_encoding_seq(tweet_word_list, mentions)
    
# #     print('tallying list lengths: ',len(tweet_sentID_list),len(tweet_word_list),len(tweet_encoding_list))
    
#     for encoded_list_index, sentID in enumerate(tweet_sentID_list):
#         encoded_df_dict={'Sentence #':tweet_sentID_list[encoded_list_index], 'Word':tweet_word_list[encoded_list_index], 'Tag':tweet_encoding_list[encoded_list_index]}
#         encoded_df_holder.append(encoded_df_dict)
# #         file_text+=tweet_word_list[encoded_list_index]+'\t'+tweet_encoding_list[encoded_list_index]+'\n'
# #     file_text+='\n'

# encoded_df_out=pd.DataFrame(encoded_df_holder,columns=encoded_df_columns)
# print('post-encoding dataframe: ', len(encoded_df_out))


In [42]:
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/tweets_3k_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/venezuela_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/billnye_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/pikapika_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/ripcity_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/roevwade_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)
# encoded_df_out.to_csv("/Users/satadisha/Documents/GitHub/billdeblasio_BIOannotated_twokenized.csv", sep=',', encoding='utf-8',index=False)


# print(encoded_df_out.tail(40))

# import re
# mystr='Macron.'
# print(split_apostrophe(mystr))
# print(mystr.split('-'))
# re.split("(-)",mystr)
# if((mystr.count('.')==1)&(mystr.endswith('.'))):
#     temp=list(filter(lambda elem: elem!='',re.split("(\.)",mystr)))
#     print(temp)

In [68]:
# conll_results=[result_line.split('\t') for result_line in result.split('\n')]
# conll_nounPhrase_chunking(conll_results)

def getConnectedComponents(visited, adjList):
    cc=[]
    cc_positions=[]
    nodeList=list(visited.keys())
#     print('**',visited,adjList)
    for ind in range(len(nodeList)):
        node=nodeList[ind]
#         print('==>',node)
        if not(visited[node][0]):
            if(ind>0):
                last.sort(key = int)
                if('^' in posStr):
#                     print('::',last)
                    candidateStringInner=(' '.join([visited[elem][1] for elem in last])).strip()
                    cc.append(candidateStringInner)
                    cc_positions.append(last)
            last=[]
            posStr=''
            bfs=[node]
            while(bfs):
                curr=bfs.pop(0)
                visited[curr][0]=True
                last.append(curr)
                posStr+=visited[curr][2]
                for neighbour in adjList[curr]:
                    if(not visited[neighbour][0]):
                        bfs.append(neighbour)
        ind+=1
    last.sort(key = int)
    if('^' in posStr):
        candidateString=(' '.join([visited[elem][1] for elem in last])).strip()
        cc.append(candidateString)
        cc_positions.append(last)
#     print('connected components:')
#     print(cc)
    return cc, cc_positions

def conll_nounPhrase_chunking(tabbed_entries):
    spans=[]
    span=[]
    for tabbed_entry in tabbed_entries:
        entry=[]
        if((tabbed_entry[3]==proper_noun_tag)|(tabbed_entry[3]==common_noun_tag)):
            entry=tabbed_entry
        if(tabbed_entry[3]==prep_tag):
            head=int(tabbed_entry[6])-1
            if(head>0):
                head_entry=tabbed_entries[head]
                if((head_entry[3]==proper_noun_tag)|(head_entry[3]==common_noun_tag)):
                    entry=tabbed_entry
        if(entry):
            if(int(entry[0])>1):
                if(span):
                    if((int(entry[0])-int(span[-1][0]))>1):
                        spans.append(span)
                        span=[entry]
                    else:
                        span.append(entry)
                else:
                    span=[entry]
            else:
                span=[entry]
    if(spans):
        if(spans[-1][0]!=span[0]):
            spans.append(span)
    else:
        if(span):
            spans.append(span)
    
    final_spans=[]
    final_spans_positions=[]
    for span in spans:
        minIndex=int(span[0][0])
        maxIndex=int(span[-1][0])
        visited={}
        adjList={}
        for entry in span:
            visited[entry[0]]=[False,entry[1],entry[3]]
            if(entry[0] not in adjList.keys()):
                adjList[entry[0]]=[]
            dependency=entry[6]
            if((int(dependency)>=minIndex)&(int(dependency)<=maxIndex)):
                adjList[entry[0]].append(dependency)
                if(dependency not in adjList.keys()):
                    adjList[dependency]=[]
                adjList[dependency].append(entry[0])
        retTup=getConnectedComponents(visited,adjList)
        final_spans.extend(retTup[0])
        final_spans_positions.extend(retTup[1])
    return final_spans,final_spans_positions

In [24]:
# Uncomment when printing outputs

# f1 = open("/Users/satadisha/Documents/GitHub/btc.emerging.test.conll.preproc.url", "w")
# file_text1=''

# f = open("/Users/satadisha/Documents/GitHub/btc.emerging.test.conll.preproc.url.postag", "w")
# file_text=''

# candidates=[]
# sentId=0
# # tweetId=0

# for sentence in sentenceList:
# # for tweet in tweetList:
#     result=result_conll[sentId]
# #     result=result_conll[tweetId]
#     tweet_word_list=[]
#     conll_results=[result_line.split('\t') for result_line in result.split('\n')]
#     for result_line in conll_results:
#         file_text+=result_line[1]+'\t'+result_line[3]+'\n'
#         tweet_word_list+=result_line[1]
#         file_text1+=result_line[1]+'\tO'+'\n'
# #         print('result_line: ', result_line)
#     file_text+='\n'
# #     tweetId+=1
# #     tweetMentions=mentionList[tweetId]
# #     tweet_encoding_list= get_encoding_seq(tweet_word_list, tweetMentions)
# #     print(len(tweet_word_list),len(tweet_encoding_list))
# #     for ind, word in enumerate(tweet_word_list):
# #         file_text1+=word+'\t'+tweet_encoding_list[ind]+'\n'
#     file_text1+='\n'
#     sentId+=1
# #     tweetId+=1

# f.write(file_text)
# f.close()

# f1.write(file_text1)
# f1.close()

In [69]:
candidates=[]
sentId=0
CTrie=trie.Trie("ROOT")
for sentence in sentenceList:
    result=result_conll[sentId]
    conll_results=[result_line.split('\t') for result_line in result.split('\n')]
    sentence_candidates,sentence_candidates_positions=conll_nounPhrase_chunking(conll_results)
#     print(sentence)
#     print(conll_results)
#     print(sentence_candidates)

    candidate_ind=0
    phase1Out=''
#     for candidate in ne_List_allCheck:
#         position = '*'+'*'.join(str(v) for v in candidate.position)
#         position=position+'*'
#         candidate.set_sen_index(sen_index)
#         phase1Out+=(((candidate.phraseText).lstrip(string.punctuation)).strip())+ '::'+str(position)+"||" 

    for candidateText in sentence_candidates:
#         print(candidateText)
        candidateText=candidateText.lower()
        position = '*'+'*'.join(str(v) for v in sentence_candidates_positions[candidate_ind])
        position=position+'*'
#         print(candidateText,sentence_candidates_positions[candidate_ind])
        CTrie.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)
        candidate_ind+=1
        phase1Out+=(((candidateText).lstrip(string.punctuation)).strip())+ '::'+str(position)+"||" 
        
#     candidates.append(sentence_candidates)
    candidates.append(phase1Out)

    sentId+=1
#     print('===========')
print(len(sentenceList),len(tweets_unpartitoned),len(candidates))

tweet_sentence_df['phase1Candidates']=candidates

# print(tweet_sentence_df['phase1Candidates'].tolist())

15036 9338 15036


In [70]:
# print(tweet_sentence_df['phase1Candidates'].tolist())

In [71]:
candidates=CTrie.displayTrie("",[])
print(len(candidates))

npchunker_phaseI_time_out = time.time()

print("NP Chunker time",(npchunker_phaseI_time_out-npchunker_phaseI_time_in))

9839
NP Chunker time 327.9848680496216


In [20]:
# import numpy as np
# mylist =[0,0,0,0,0]
# mylist[1:-1]=np.add(mylist[1:-1],[1,2,3]).tolist()
# mylist[1:-1]=np.add(mylist[1:-1],[1,2,3]).tolist()
# print(mylist)

In [30]:


import phase2_Trie_baseline_reintroduction_effectiveness as phase2

z_score=-0.1119
max_batch_value=0
phase2stopwordList=[]
reintroduction_threshold_dummy=2


In [31]:
Phase2 = phase2.EntityResolver()

time_in = time.time()
candidate_base_post_Phase2, converted_candidates, complete_tweet_dataframe_grouped_df_sorted= Phase2.executor(max_batch_value,tweet_sentence_df,CTrie,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweets_unpartitoned)
time_out = time.time()

print('time taken',(time_out-time_in))

device is cpu
ambiguous_candidates_in_batch:  0
dataframe lengths:  15036 15036 8064
-0.18754906049338768
torch.Size([8064, 1])
torch.Size([8064])
0.09069839864969254 0.9531461596488953
good_candidates 4816
ambiguous_candidates 1731
bad_candidates 1517


/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/phase2_Trie_baseline_reintroduction_effectiveness.py:1338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']>=0.67]='g'
/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/phase2_Trie_baseline_reintroduction_effectiveness.py:1339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][(candidate_featureBase_DF['probability'] > 0.4) & (candidate_featureBase_DF['probability'] < 0.67)] = 'a'
/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/

completed tweets:  13583 incomplete tweets:  1453
16
16
final tally:  15036 15036
524:  524    [[]]
Name: output_mentions, dtype: object
['tweetID', 'index', 'entry_batch', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates', 'annotation', 'stanford_candidates', 'output_mentions', 'completeness', 'current_minus_entry', 'candidates_with_label', 'only_good_candidates', 'ambiguous_candidates']
time taken 15.749476909637451


In [32]:
# tweets_unpartitoned.head()

In [33]:
# complete_tweet_dataframe_grouped_df_sorted.head(20)

In [34]:
# from ast import literal_eval
true_positive_count=0
false_positive_count=0
false_negative_count=0

total_annotations=0
total_tagged=0

for index, row in tweets_unpartitoned.iterrows():
    unrecovered_annotated_mention_list=[]
    tp_counter_inner=0
    fp_counter_inner=0
    fn_counter_inner=0
    
    tweet_ID=row['ID']
    annotated_mention_list=[]
    annotated=row['mentions_other'].lower()
    
    if(annotated):
        tweet_level=annotated.split(';')
        if(tweet_level):
            tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
            for elem in tweet_level:
                sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
                if(sentence_level):
                    annotated_mention_list.extend([innermost.strip() for innermost in sentence_level if(innermost)])
    
#     output_mentions_list=flatten(complete_tweet_dataframe_grouped_df_sorted[complete_tweet_dataframe_grouped_df_sorted.tweetID==tweet_ID].output_mentions.tolist(),[])
    output_mentions_list=flatten(complete_tweet_dataframe_grouped_df_sorted[complete_tweet_dataframe_grouped_df_sorted.tweetID==tweet_ID].phase1Candidates.tolist(),[])
    
    #print(row['TweetText'])
    #print(tweet_ID, annotated_mention_list)
    #print(output_mentions_list)
    
    all_postitive_counter_inner=len(output_mentions_list)
    total_tagged+=len(output_mentions_list)
    total_annotations+=len(annotated_mention_list)
    
    while(annotated_mention_list):
        if(len(output_mentions_list)):
            annotated_candidate= annotated_mention_list.pop()
            if(annotated_candidate in output_mentions_list):
                output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                tp_counter_inner+=1
            else:
                unrecovered_annotated_mention_list.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list.extend(annotated_mention_list)
            break

    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner=len(unrecovered_annotated_mention_list)
    fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

#     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

    true_positive_count+=tp_counter_inner
    false_positive_count+=fp_counter_inner
    false_negative_count+=fn_counter_inner

print(total_annotations,total_tagged)
print(true_positive_count,false_positive_count,false_negative_count)

        

10884 12993
6092 6901 4792


In [35]:
precision=(true_positive_count)/(true_positive_count+false_positive_count)
recall=(true_positive_count)/(true_positive_count+false_negative_count)
f_measure=2*(precision*recall)/(precision+recall)
print(precision,recall,f_measure)

0.46886785192026476 0.559720690922455 0.5102818612053441


In [55]:
#SVM
2425 487 905
0.832760989010989 0.7282282282282282 0.7769945530278757

#RF
2425 487 905
0.832760989010989 0.7282282282282282 0.7769945530278757

#LR
2415 460 915
0.84 0.7252252252252253 0.7784045124899275

SyntaxError: invalid syntax (<ipython-input-55-a150bcecdd2b>, line 2)

# Ritter- TwitterNLP in Phase 1


In [45]:
#3k annotated tweets
import phase2_Trie_baseline_reintroduction_effectiveness as phase2
from ast import literal_eval
import string

z_score=-0.1119
max_batch_value=0
phase2stopwordList=[]
reintroduction_threshold_dummy=2

def remAmpersand(candidateStr):
    candidateStr=candidateStr.replace('&amp;','')
    return candidateStr
    
string.punctuation=string.punctuation+'…‘’'

## these are ritter phase I results
# f = open("/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/candidatesCovidRitter.txt",'r')
# f = open("/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/tweets_3k_annotated.txt",'r')
f = open("/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/candidatesBTCRitter.txt",'rb')


# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter_tweets_3k_annotated_output.csv",sep =',',keep_default_na=False)
# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-venezuela-output.csv",sep =',',keep_default_na=False)

# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-roevwade-output.csv",sep =',',keep_default_na=False)
# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-ripcity-output.csv",sep =',',keep_default_na=False)
# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-pikapika-output.csv",sep =',',keep_default_na=False)
# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-billnye-output.csv",sep =',',keep_default_na=False)
# ritter_annotator=pd.read_csv("/Users/satadisha/Documents/GitHub/ritter-baseline-setup/ritter-billdeblasio-output.csv",sep =',',keep_default_na=False)

In [46]:
# tweets=tweets_unpartitoned['TweetText'].tolist()
# print(len(ritter_annotator))
# print(ritter_annotator.columns.tolist())
CTrie_ritter=trie.Trie("ROOT")
tweet_sentence_df_copy=tweet_sentence_df.copy(deep=True)

In [47]:
# ritter_annotated_candidates=ritter_annotator['Output'].tolist()
# for candidate in ritter_annotated_candidates:
#     candidateList= [remAmpersand(elem).strip(string.punctuation).strip() for elem in candidate.lower().split(',') if(elem)]
# #     print(candidateList)
#     for candidateText in candidateList:
#         CTrie_ritter.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)

ritterPhaseIcandidates=[]
file_text=f.read().decode('ISO-8859-1')
# output_sentences=list(filter (lambda elem: elem!='', file_text.split('\n'))) #conll
output_sentences=file_text.split('\n')
print(len(output_sentences))
ritter_annotated_candidates=[]
for line in output_sentences:
    line_candidates = []
    if(line):
        tabs=line.split('\t')
        if(tabs):
            for candidate in tabs:
                candidate=candidate.strip().lower()
                ritter_annotated_candidates.append(candidate)
                if(candidate):
                    line_candidates.append(candidate)
                CTrie_ritter.__setitem__(candidate.split(),len(candidate.split()),[],batch_number)
    ritterPhaseIcandidates.append(line_candidates)
print(len(ritter_annotated_candidates))
print(len(ritterPhaseIcandidates))
candidatesinRitterTrie=CTrie_ritter.displayTrie("",[])
print(len(candidatesinRitterTrie))


# tweet_sentence_df_copy['phase1Candidates']=ritter_annotated_candidates

9339
6790
9339
4440


In [ ]:
# from ast import literal_eval
true_positive_count=0
false_positive_count=0
false_negative_count=0

total_annotations=0
total_tagged=0

for index, row in tweets_unpartitoned.iterrows():
    unrecovered_annotated_mention_list=[]
    tp_counter_inner=0
    fp_counter_inner=0
    fn_counter_inner=0
    
    tweet_ID=row['ID']
    annotated_mention_list=[]
    annotated=row['mentions_other'].lower()
    
    if(annotated):
        tweet_level=annotated.split(';')
        if(tweet_level):
            tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
            for elem in tweet_level:
                sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
                if(sentence_level):
                    annotated_mention_list.extend([innermost.strip() for innermost in sentence_level if(innermost)])
    
#     output_mentions_list=flatten(complete_tweet_dataframe_grouped_df_sorted[complete_tweet_dataframe_grouped_df_sorted.tweetID==tweet_ID].output_mentions.tolist(),[])
    output_mentions_list=ritterPhaseIcandidates[index]
    
    #print(row['TweetText'])
    print(tweet_ID, annotated_mention_list)
    print(output_mentions_list)
    
    all_postitive_counter_inner=len(output_mentions_list)
    total_tagged+=len(output_mentions_list)
    total_annotations+=len(annotated_mention_list)
    
    while(annotated_mention_list):
        if(len(output_mentions_list)):
            annotated_candidate= annotated_mention_list.pop()
            if(annotated_candidate in output_mentions_list):
                output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                tp_counter_inner+=1
            else:
                unrecovered_annotated_mention_list.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list.extend(annotated_mention_list)
            break

    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner=len(unrecovered_annotated_mention_list)
    fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

#     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

    true_positive_count+=tp_counter_inner
    false_positive_count+=fp_counter_inner
    false_negative_count+=fn_counter_inner

print(total_annotations,total_tagged)
print(true_positive_count,false_positive_count,false_negative_count)

precision=(true_positive_count)/(true_positive_count+false_positive_count)
recall=(true_positive_count)/(true_positive_count+false_negative_count)
f_measure=2*(precision*recall)/(precision+recall)
print(precision,recall,f_measure)

In [48]:
print(candidatesinRitterTrie)

['tosca', 'gateshead', 'dorothy gale', 'oz', 'tantamounttotreaser', 'td', 'td chargers', 'td bank', 'facebook', 'facebook timeline resources', 'facebook ipo', 'salmon valley', 'edgar promo', 'pride and prejudice', 'otis', 'dad', 'kane', 'florida', 'florida gas plant rock', 'daniel', 'daniel james evans', 'daniel jones', 'daniel sturridge', 'daniel ellsberg', 'daniel morcombe', 'daniel radcliffe', 'daniel finkelstein', 'woolwich room', 'database', 'oracle', 'mysql', 'perl', 'java', 'london', 'london da', 'london java developer', 'london college of fashion ma show', 'london king', 'london street', 'yorkshire- doncaster', 'leeds', 'god', 'god cora', 'god and gosling', 'liverpool', 'liverpool 1', 'liverpool xi', 'liverpool city council', 'kuyt', 'louise', 'alan', 'alan b', 'alan turing', 'alan partridge', 'alan yentob', 'alan porter', 'alan pardew', 'twitter', 'hackney', 'uk', 'uk north sea', 'uk university', 'uk tour', 'uk 6 germany', 'uk gov', 'uk embassy', 'uk mps', 'uk foreign secretar

In [49]:
Phase2_w_Ritter = phase2.EntityResolver()

time_in = time.time()
candidate_base_post_Phase2_w_Ritter, converted_candidates_w_Ritter, complete_tweet_dataframe_grouped_df_sorted_w_Ritter= Phase2_w_Ritter.executor(max_batch_value,tweet_sentence_df,CTrie_ritter,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweet_sentence_df_copy)
time_out = time.time()

print(time_out-time_in)


device is cpu
Input Shape:  torch.Size([679, 7])
Output Shape:  torch.Size([679])
ambiguous_candidates_in_batch:  0
dataframe lengths:  15036 15036 4202
-0.2298890856774015
torch.Size([4202, 1])
torch.Size([4202])
0.010653595440089703 0.9302037358283997
good_candidates 3588
ambiguous_candidates 221
bad_candidates 393


/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/phase2_Trie_baseline_reintroduction_effectiveness.py:1338: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']>=0.67]='g'
/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/phase2_Trie_baseline_reintroduction_effectiveness.py:1339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][(candidate_featureBase_DF['probability'] > 0.4) & (candidate_featureBase_DF['probability'] < 0.67)] = 'a'
/Users/satadisha/Documents/GitHub/tweebo-parser-for-preprocess/

completed tweets:  14487 incomplete tweets:  549
16
16
final tally:  15036 15036
524:  524    [[]]
Name: output_mentions, dtype: object
['tweetID', 'index', 'entry_batch', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates', 'annotation', 'stanford_candidates', 'output_mentions', 'completeness', 'current_minus_entry', 'candidates_with_label', 'only_good_candidates', 'ambiguous_candidates']
10.26089596748352


In [50]:
# complete_tweet_dataframe_grouped_df_sorted_w_Ritter.head(20)

In [51]:
# from ast import literal_eval
true_positive_count_ritter=0
false_positive_count_ritter=0
false_negative_count_ritter=0

total_annotations_ritter=0
total_tagged_ritter=0
all_annotations=[]

for index, row in tweets_unpartitoned.iterrows():
    unrecovered_annotated_mention_list_ritter=[]
    tp_counter_inner_ritter=0
    fp_counter_inner_ritter=0
    fn_counter_inner_ritter=0
    
    tweet_ID=row['ID']
    annotated_mention_list_ritter=[]
    annotated=row['mentions_other'].lower()
    
    if(annotated):
        tweet_level=annotated.split(';')
        if(tweet_level):
            tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
            for elem in tweet_level:
                sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
                if(sentence_level):
                    annotated_mention_list_ritter.extend([innermost.strip() for innermost in sentence_level if(innermost)])
#     output_mentions_list_ritter=flatten(complete_tweet_dataframe_grouped_df_sorted_w_Ritter[complete_tweet_dataframe_grouped_df_sorted_w_Ritter.tweetID==tweet_ID].phase1Candidates.tolist(),[])
    output_mentions_list_ritter=flatten(complete_tweet_dataframe_grouped_df_sorted_w_Ritter[complete_tweet_dataframe_grouped_df_sorted_w_Ritter.tweetID==tweet_ID].output_mentions.tolist(),[])
    
#     print(row['TweetText'])
#     print(tweet_ID, annotated_mention_list_ritter)
#     print(output_mentions_list_ritter)
    all_annotations+=annotated_mention_list_ritter
    
    all_postitive_counter_inner_ritter=len(output_mentions_list_ritter)
    total_tagged_ritter+=len(output_mentions_list_ritter)
    total_annotations_ritter+=len(annotated_mention_list_ritter)
    
    while(annotated_mention_list_ritter):
        if(len(output_mentions_list_ritter)):
            annotated_candidate= annotated_mention_list_ritter.pop().lower()
            if(annotated_candidate in output_mentions_list_ritter):
                output_mentions_list_ritter.pop(output_mentions_list_ritter.index(annotated_candidate))
                tp_counter_inner_ritter+=1
            else:
                unrecovered_annotated_mention_list_ritter.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list_ritter.extend(annotated_mention_list_ritter)
            break

    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner_ritter=len(unrecovered_annotated_mention_list_ritter)
    fp_counter_inner_ritter=all_postitive_counter_inner_ritter- tp_counter_inner_ritter

#     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

    true_positive_count_ritter+=tp_counter_inner_ritter
    false_positive_count_ritter+=fp_counter_inner_ritter
    false_negative_count_ritter+=fn_counter_inner_ritter

print(total_annotations_ritter,total_tagged_ritter)
print(true_positive_count_ritter,false_positive_count_ritter,false_negative_count_ritter)


10884 6508
4795 1713 6089


In [52]:
print(len(set(all_annotations)))

7322


In [53]:
precision_ritter=(true_positive_count_ritter)/(true_positive_count_ritter+false_positive_count_ritter)
recall_ritter=(true_positive_count_ritter)/(true_positive_count_ritter+false_negative_count_ritter)
f_measure_ritter=2*(precision_ritter*recall_ritter)/(precision_ritter+recall_ritter)
print(precision_ritter,recall_ritter,f_measure_ritter)

0.7367854947756607 0.4405549430356487 0.5514029438822446


#SVM
0.8605577689243028 0.8428520752039731 0.8516129032258064

#RF
0.8614435981138919 0.8433948863636364 0.8523237035707877

#LR
0.8610108303249098 0.8451452870304749 0.8530042918454936

# Gaguilar as Phase I

In [17]:
import phase2_Trie_baseline_reintroduction_effectiveness as phase2
from ast import literal_eval
import string

z_score=-0.1119
max_batch_value=0
phase2stopwordList=[]
reintroduction_threshold_dummy=2



def remAmpersand(candidateStr):
    candidateStr=candidateStr.replace('&amp;','')
    return candidateStr
    
string.punctuation=string.punctuation+'…‘’'
f = open("/Users/satadisha/Documents/GitHub/covid_2K.txt",'r')

In [18]:
CTrie_gaguilar=trie.Trie("ROOT")
tweet_sentence_df_copy=tweet_sentence_df.copy(deep=True)

In [19]:

Phase2_w_gaguilar = phase2.EntityResolver()
file_text=f.read()
output_sentences=list(filter (lambda elem: elem!='', file_text.split('\n'))) #conll
print(len(output_sentences))
gaguilar_annotated_candidates=[]
for line in output_sentences:
    if(line):
#         tabs=line.split('\t')
#         if(tabs):
#             for candidate in tabs:
        gaguilar_annotated_candidates.append(line)
print(len(gaguilar_annotated_candidates))

337
337


/Users/satadisha/Documents/GitHub/tweebo-parser/SVM.py:201: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.clf.fit(self.trainArr, self.trainRes)


In [20]:
for candidate in gaguilar_annotated_candidates:
    candidateList= [remAmpersand(elem).strip(string.punctuation).strip() for elem in candidate.lower().split(',') if(elem)]
#     print(candidateList)
    for candidateText in candidateList:
        CTrie_gaguilar.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)

candidatesingaguilarTrie=CTrie_gaguilar.displayTrie("",[])
print(len(candidatesingaguilarTrie))

337


In [21]:

candidate_base_post_Phase2_w_gaguilar, converted_candidates_w_gaguilar, complete_tweet_dataframe_grouped_df_sorted_w_gaguilar= Phase2_w_gaguilar.executor(max_batch_value,tweet_sentence_df,CTrie_gaguilar,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweet_sentence_df_copy)


ambiguous_candidates_in_batch:  0
dataframe lengths:  3091 3091 328
-0.07019717286748468
For entities:  (246, 6)
For non-entities:  (53, 6)
For ambiguous:  (29, 6)
For entities:  (246, 6)
For non-entities:  (53, 6)


/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:1319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']>=0.67]='g'
/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:1320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][(candidate_featureBase_DF['probability'] > 0.4) & (candidate_featureBase_DF['probability'] < 0.67)] = 'a'
/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effective

For ambiguous:  (29, 6)


/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:370: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ambiguous_bad_candidates['max_column'] =ambiguous_bad_candidates[['cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']].idxmax(axis=1)


completed tweets:  3024 incomplete tweets:  67
16
16
final tally:  3091 3091
524:  524    [[]]
Name: output_mentions, dtype: object
['tweetID', 'index', 'entry_batch', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates', 'annotation', 'stanford_candidates', 'output_mentions', 'completeness', 'current_minus_entry', 'candidates_with_label', 'only_good_candidates', 'ambiguous_candidates']


In [22]:
# from ast import literal_eval
true_positive_count_gaguilar=0
false_positive_count_gaguilar=0
false_negative_count_gaguilar=0

total_annotations_gaguilar=0
total_tagged_gaguilar=0

all_annotated=0

for index, row in tweets_unpartitoned.iterrows():
    unrecovered_annotated_mention_list_gaguilar=[]
    tp_counter_inner_gaguilar=0
    fp_counter_inner_gaguilar=0
    fn_counter_inner_gaguilar=0
    
    tweet_ID=row['ID']
    annotated_mention_list_gaguilar=[]
    annotated=row['mentions_other'].lower()
    
    if(annotated):
        tweet_level=annotated.split(';')
        if(tweet_level):
            tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
            for elem in tweet_level:
                sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
                if(sentence_level):
                    annotated_mention_list_gaguilar.extend([innermost.strip() for innermost in sentence_level if(innermost)])
    output_mentions_list_gaguilar=flatten(complete_tweet_dataframe_grouped_df_sorted_w_gaguilar[complete_tweet_dataframe_grouped_df_sorted_w_gaguilar.tweetID==tweet_ID].output_mentions.tolist(),[])
    
#     print(tweet_ID, row['TweetText'])
#     print(annotated_mention_list_gaguilar)
    all_annotated+=len(annotated_mention_list_gaguilar)
#     print(output_mentions_list_gaguilar)
    
    all_postitive_counter_inner_gaguilar=len(output_mentions_list_gaguilar)
    total_tagged_gaguilar+=len(output_mentions_list_gaguilar)
    total_annotations_gaguilar+=len(annotated_mention_list_gaguilar)
    
    while(annotated_mention_list_gaguilar):
        if(len(output_mentions_list_gaguilar)):
            annotated_candidate= annotated_mention_list_gaguilar.pop().lower()
            if(annotated_candidate in output_mentions_list_gaguilar):
                output_mentions_list_gaguilar.pop(output_mentions_list_gaguilar.index(annotated_candidate))
                tp_counter_inner_gaguilar+=1
            else:
                unrecovered_annotated_mention_list_gaguilar.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list_gaguilar.extend(annotated_mention_list_gaguilar)
            break

    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner_gaguilar=len(unrecovered_annotated_mention_list_gaguilar)
    fp_counter_inner_gaguilar=all_postitive_counter_inner_gaguilar- tp_counter_inner_gaguilar

#     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

    true_positive_count_gaguilar+=tp_counter_inner_gaguilar
    false_positive_count_gaguilar+=fp_counter_inner_gaguilar
    false_negative_count_gaguilar+=fn_counter_inner_gaguilar
    

print(total_annotations_gaguilar,total_tagged_gaguilar)
print(true_positive_count_gaguilar,false_positive_count_gaguilar,false_negative_count_gaguilar)


0 In an effort to provide more space for social distancing, the city is closing streets to cars starting Friday.
[]
[]
1 The public did not follow social distancing, outdoor groupings restricted, however businesses will adhere aye right !!!!
[]
[]
2 Social distancing is oddly having a therapeutic effect on me. I could do this all the time. Outside of maybe 5 peop…
[]
[]
3 Is he nuts? How is this social distancing?
[]
[]
4 @JuliaDavisNews @MaxBoot I am willing to sacrifice my nails and hair for social distancing & saving lives. Besides…
[]
[]
5 Social distancing-quarantine got me like...
[]
[]
6 People of all ages can use technology to reach out to loved ones while social distancing. Learn more
[]
[]
7 Why are non key workers still being allowed to crowd onto public transport?
[]
[]
8 My girls refuse to maintain social distancing... #PhysicalDistancing #FlattenTheCurve
[]
[]
9 All this means is people still aren’t washing their hands, social distancing and everything else ?
[]
[]
10 @Ke

In [23]:
print(all_annotated)
precision_gaguilar=(true_positive_count_gaguilar)/(true_positive_count_gaguilar+false_positive_count_gaguilar)
recall_gaguilar=(true_positive_count_gaguilar)/(true_positive_count_gaguilar+false_negative_count_gaguilar)
f_measure_gaguilar=2*(precision_gaguilar*recall_gaguilar)/(precision_gaguilar+recall_gaguilar)
print(precision_gaguilar,recall_gaguilar,f_measure_gaguilar)

764
0.7154308617234469 0.46727748691099474 0.5653206650831354


#LR
0.8313738663083641 0.8384146341463414 0.8348794063079776

#RF
0.8313738663083641 0.8384146341463414 0.8348794063079776

#SVM
0.8318821165438715 0.8406091370558376 0.8362228581046962

# Just NeuroNER

In [85]:
# true_positive_count_neuroner=0
# false_positive_count_neuroner=0
# false_negative_count_neuroner=0

# total_annotations_neuroner=0
# total_tagged_neuroner=0

# neuroner_file = open('mentions_output_tweets_3K.txt', 'r') 
# neuroner_lines = neuroner_file.readlines()
# print(len(neuroner_lines))
# line_count=0
# neuroner_annotated_candidates=[]

# for index, row in tweets_unpartitoned.iterrows():
#     unrecovered_annotated_mention_list_neuroner=[]
#     tp_counter_inner_neuroner=0
#     fp_counter_inner_neuroner=0
#     fn_counter_inner_neuroner=0
    
#     annotated_mention_list_neuroner=[]
#     annotated=row['mentions_other'].lower()
    
#     if(annotated):
#         tweet_level=annotated.split(';')
#         if(tweet_level):
#             tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
#             for elem in tweet_level:
#                 sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
#                 if(sentence_level):
#                     annotated_mention_list_neuroner.extend([innermost.strip() for innermost in sentence_level if(innermost)])
                    
#     neuroner_output=neuroner_lines[line_count]
#     output_mentions_list_neuroner=[candidate.lower().strip(string.punctuation).strip() for candidate in neuroner_output.split(',') if(candidate.strip(string.punctuation).strip())]
#     neuroner_annotated_candidates.extend(output_mentions_list_neuroner)
    
#     all_postitive_counter_inner_neuroner=len(output_mentions_list_neuroner)
#     total_tagged_neuroner+=len(output_mentions_list_neuroner)
#     total_annotations_neuroner+=len(annotated_mention_list_neuroner)
    
#     while(annotated_mention_list_neuroner):
#         if(len(output_mentions_list_neuroner)):
#             annotated_candidate= annotated_mention_list_neuroner.pop()
#             if(annotated_candidate in output_mentions_list_neuroner):
#                 output_mentions_list_neuroner.pop(output_mentions_list_neuroner.index(annotated_candidate))
#                 tp_counter_inner_neuroner+=1
#             else:
#                 unrecovered_annotated_mention_list_neuroner.append(annotated_candidate)
#         else:
#             unrecovered_annotated_mention_list_neuroner.extend(annotated_mention_list)
#             break

#     # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
#     fn_counter_inner_neuroner=len(unrecovered_annotated_mention_list_neuroner)
#     fp_counter_inner_neuroner=all_postitive_counter_inner_neuroner - tp_counter_inner_neuroner

# #     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

#     true_positive_count_neuroner+=tp_counter_inner_neuroner
#     false_positive_count_neuroner+=fp_counter_inner_neuroner
#     false_negative_count_neuroner+=fn_counter_inner_neuroner
    
#     line_count+=1
    
# print(total_annotations_neuroner,total_tagged_neuroner)
# print(true_positive_count_neuroner,false_positive_count_neuroner,false_negative_count_neuroner)    

3068
3374 3849
2501 1348 673


In [86]:
# precision_neuroner=(true_positive_count_neuroner)/(true_positive_count_neuroner+false_positive_count_neuroner)
# recall_neuroner=(true_positive_count_neuroner)/(true_positive_count_neuroner+false_negative_count_neuroner)
# f_measure_neuroner=2*(precision_neuroner*recall_neuroner)/(precision_neuroner+recall_neuroner)
# print(precision_neuroner,recall_neuroner,f_measure_neuroner)

0.6497791634190699 0.7879647132955262 0.7122312402107361


# NeuroNER as Phase I Entity Tagger

In [87]:
# tweet_sentence_df_2nd_copy=tweet_sentence_df.copy(deep=True)
# CTrie_neuroner=trie.Trie("ROOT")
# print(len(neuroner_annotated_candidates))
# print(phase2stopwordList)

# for candidateText in neuroner_annotated_candidates:
# #     print(candidateText)
#     if(candidateText not in all_stopwords):
#         CTrie_neuroner.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)

# candidatesinNeuronerTrie=CTrie_neuroner.displayTrie("",[])
# print(len(candidatesinNeuronerTrie))

3849
[]
1425


In [93]:
# Phase2_w_Neuroner = phase2.EntityResolver()
# candidate_base_post_Phase2_w_Neuroner, converted_candidates_w_Neuroner, complete_tweet_dataframe_grouped_df_sorted_w_Neuroner= Phase2_w_Neuroner.executor(max_batch_value,tweet_sentence_df_2nd_copy,CTrie_neuroner,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweet_sentence_df_2nd_copy)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ambiguous_candidates_in_batch:  0
dataframe lengths:  4721 4721 1057
-0.2661675732774245


/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:1282: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][candidate_featureBase_DF['probability']>=0.8]='g'
/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:1283: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_featureBase_DF['status'][(candidate_featureBase_DF['probability'] > 0.4) & (candidate_featureBase_DF['probability'] < 0.8)] = 'a'
/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectivene

For entities:  (797, 6)
For non-entities:  (216, 6)
For ambiguous:  (44, 6)
For entities:  (797, 6)
For non-entities:  (216, 6)
For ambiguous:  (44, 6)
Empty DataFrame
Columns: [candidate, batch, length, cap, substring-cap, s-o-sCap, all-cap, non-cap, non-discriminative, cumulative, Z_ScoreUnweighted, normalized_cap, normalized_capnormalized_substring-cap, normalized_s-o-sCap, normalized_all-cap, normalized_non-cap, normalized_non-discriminative, probability, status]
Index: []


/Users/satadisha/Documents/GitHub/tweebo-parser/phase2_Trie_baseline_reintroduction_effectiveness.py:369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ambiguous_bad_candidates['max_column'] =ambiguous_bad_candidates[['cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']].idxmax(axis=1)


completed tweets:  4525 incomplete tweets:  196
16
16
final tally:  4721 4721
524:  524    [[world news, fbi, fisa, trump, washington post]]
Name: output_mentions, dtype: object
['tweetID', 'index', 'entry_batch', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates', 'annotation', 'stanford_candidates', 'output_mentions', 'completeness', 'current_minus_entry', 'candidates_with_label', 'only_good_candidates', 'ambiguous_candidates']


In [94]:
# complete_tweet_dataframe_grouped_df_sorted_w_Neuroner

In [95]:
# # from ast import literal_eval
# true_positive_count_neuroner=0
# false_positive_count_neuroner=0
# false_negative_count_neuroner=0

# total_annotations_neuroner=0
# total_tagged_neuroner=0

# for index, row in tweets_unpartitoned.iterrows():
#     unrecovered_annotated_mention_list_neuroner=[]
#     tp_counter_inner_neuroner=0
#     fp_counter_inner_neuroner=0
#     fn_counter_inner_neuroner=0
    
#     tweet_ID=row['ID']
#     annotated_mention_list_neuroner=[]
#     annotated=row['mentions_other'].lower()
    
#     if(annotated):
#         tweet_level=annotated.split(';')
#         if(tweet_level):
#             tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
#             for elem in tweet_level:
#                 sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
#                 if(sentence_level):
#                     annotated_mention_list_neuroner.extend([innermost.strip() for innermost in sentence_level if(innermost)])
#     output_mentions_list_neuroner=flatten(complete_tweet_dataframe_grouped_df_sorted_w_Neuroner[complete_tweet_dataframe_grouped_df_sorted_w_Neuroner.tweetID==tweet_ID].output_mentions.tolist(),[])
    
# #     print(row['TweetText'])
# #     print(tweet_ID, annotated_mention_list)
# #     print(output_mentions_list)
    
#     all_postitive_counter_inner_neuroner=len(output_mentions_list_neuroner)
#     total_tagged_neuroner+=len(output_mentions_list_neuroner)
#     total_annotations_neuroner+=len(annotated_mention_list_neuroner)
    
#     while(annotated_mention_list_neuroner):
#         if(len(output_mentions_list_neuroner)):
#             annotated_candidate= annotated_mention_list_neuroner.pop()
#             if(annotated_candidate in output_mentions_list_neuroner):
#                 output_mentions_list_neuroner.pop(output_mentions_list_neuroner.index(annotated_candidate))
#                 tp_counter_inner_neuroner+=1
#             else:
#                 unrecovered_annotated_mention_list_neuroner.append(annotated_candidate)
#         else:
#             unrecovered_annotated_mention_list_neuroner.extend(annotated_mention_list)
#             break

#     # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
#     fn_counter_inner_neuroner=len(unrecovered_annotated_mention_list_neuroner)
#     fp_counter_inner_neuroner=all_postitive_counter_inner_neuroner- tp_counter_inner_neuroner

# #     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

#     true_positive_count_neuroner+=tp_counter_inner_neuroner
#     false_positive_count_neuroner+=fp_counter_inner_neuroner
#     false_negative_count_neuroner+=fn_counter_inner_neuroner

# print(total_annotations_neuroner,total_tagged_neuroner)
# print(true_positive_count_neuroner,false_positive_count_neuroner,false_negative_count_neuroner)


3374 3129
2507 622 549


In [96]:
# precision_neuroner=(true_positive_count_neuroner)/(true_positive_count_neuroner+false_positive_count_neuroner)
# recall_neuroner=(true_positive_count_neuroner)/(true_positive_count_neuroner+false_negative_count_neuroner)
# f_measure_neuroner=2*(precision_neuroner*recall_neuroner)/(precision_neuroner+recall_neuroner)
# print(precision_neuroner,recall_neuroner,f_measure_neuroner)

0.8012144455097475 0.8203534031413613 0.8106709781729993


# Just Ritter

In [67]:
# from ast import literal_eval
true_positive_count_ritter=0
false_positive_count_ritter=0
false_negative_count_ritter=0

total_annotations_ritter=0
total_tagged_ritter=0

for index, row in tweets_unpartitoned.iterrows():
    unrecovered_annotated_mention_list_ritter=[]
    tp_counter_inner_ritter=0
    fp_counter_inner_ritter=0
    fn_counter_inner_ritter=0
    
    tweet_ID=row['ID']
    annotated_mention_list_ritter=[]
    annotated=row['mentions_other'].lower()
    output_mentions_list_ritter=[]
    
    if(annotated):
        tweet_level=annotated.split(';')
        if(tweet_level):
            tweet_level=[tweet_level_elem for tweet_level_elem in tweet_level if(tweet_level_elem)]
            for elem in tweet_level:
                sentence_level=[sentence_level_elem for sentence_level_elem in elem.split(',') if(sentence_level_elem)]
                if(sentence_level):
                    annotated_mention_list_ritter.extend([innermost.strip() for innermost in sentence_level if(innermost)])
                    
    candidate_list_ritter=flatten(ritter_annotator[ritter_annotator.ID==tweet_ID].Output.tolist(),[])
    for ritter_candidate in candidate_list_ritter:
        output_mentions_list_ritter+= [remAmpersand(elem).strip(string.punctuation).strip() for elem in ritter_candidate.lower().split(',') if(elem)]
        
#     output_mentions_list_ritter= [remAmpersand(elem).strip(string.punctuation).strip().lower() for elem in candidate_list_ritter]
    
    
#     print(row['TweetText'])
#     print(tweet_ID, annotated_mention_list_ritter)
#     print(output_mentions_list_ritter)
    
    all_postitive_counter_inner_ritter=len(output_mentions_list_ritter)
    total_tagged_ritter+=len(output_mentions_list_ritter)
    total_annotations_ritter+=len(annotated_mention_list_ritter)
    
    while(annotated_mention_list_ritter):
        if(len(output_mentions_list_ritter)):
            annotated_candidate= annotated_mention_list_ritter.pop()
            if(annotated_candidate in output_mentions_list_ritter):
                output_mentions_list_ritter.pop(output_mentions_list_ritter.index(annotated_candidate))
                tp_counter_inner_ritter+=1
            else:
                unrecovered_annotated_mention_list_ritter.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list_ritter.extend(annotated_mention_list)
            break

    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner_ritter=len(unrecovered_annotated_mention_list_ritter)
    fp_counter_inner_ritter=all_postitive_counter_inner_ritter- tp_counter_inner_ritter

#     print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

    true_positive_count_ritter+=tp_counter_inner_ritter
    false_positive_count_ritter+=fp_counter_inner_ritter
    false_negative_count_ritter+=fn_counter_inner_ritter

print(total_annotations_ritter,total_tagged_ritter)
print(true_positive_count_ritter,false_positive_count_ritter,false_negative_count_ritter)


3330 2095
1571 524 875


In [68]:
precision_ritter=(true_positive_count_ritter)/(true_positive_count_ritter+false_positive_count_ritter)
recall_ritter=(true_positive_count_ritter)/(true_positive_count_ritter+false_negative_count_ritter)
f_measure_ritter=2*(precision_ritter*recall_ritter)/(precision_ritter+recall_ritter)
print(precision_ritter,recall_ritter,f_measure_ritter)

0.7498806682577566 0.6422730989370401 0.6919180797181238


# Results with different annotators:

##With CS+ in phase 1:
0.7982625482625483 0.7359833877187778 0.7658589288470442

##With Turboparse chunker in phase 1:
0.8381118881118881 0.7104327208061648 0.7690086621751684

##Just TwitterNLP:
0.7460620525059666 0.6335630320226996 0.6852257781674704

##With TwitterNLP entity annotator in phase 1:
0.856 0.8288732394366197 0.8422182468694097

##Just NeuroNER:
0.6497791634190699 0.7879647132955262 0.7122312402107361

##With NeuroNER entity annotator in phase 1:
0.8012144455097475 0.8203534031413613 0.8106709781729993